<a href="https://www.kaggle.com/code/yahyasoker/poem-classification-using-bert?scriptVersionId=155310258" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install transformers > /dev/null

In [2]:
import sqlite3
import pandas as pd
import numpy as np
import os
import torch
import random
import matplotlib.pyplot as plt
import transformers
from transformers import BertTokenizer
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
#from xgboost import XGBRegressor
import lightgbm as lgb
from tqdm import tqdm
tqdm.pandas()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [3]:

# Read the training and testing datasets from CSV files
df_train = pd.read_csv('../input/poem-classification-nlp/Poem_classification - train_data.csv')
df_test = pd.read_csv('../input/poem-classification-nlp/Poem_classification - test_data.csv')
# Display the contents of the training and testing datasets
display(df_train)
display(df_test)
# Concatenate the training and testing datasets along axis 0 (row-wise)
data=pd.concat([df_train,df_test],axis=0)
display(data.info())
data=data.dropna()

,Genre,Poem
0,Music,NaN
1,Music,In the thick brushthey spend the...
2,Music,Storms are generous. ...
3,Music,—After Ana Mendieta Did you carry around the ...
4,Music,for Aja Sherrard at 20The portent may itself ...
...,...,...
836,Environment,Why make so much of fragmentary blue In here a...
837,Environment,"Woman, I wish I didn't know your name. What co..."
838,Environment,"Yonder to the kiosk, beside the creek, Paddle ..."
839,Environment,You come to fetch me from my work to-night Whe...


,Genre,Poem
0,Music,A woman walks by the bench I’m sitting onwith ...
1,Music,"Because I am a boy, the untouchability of beau..."
2,Music,"Because today we did not leave this world,We n..."
3,Music,"Big Bend has been here, been here. Shouldn’t i..."
4,Music,"I put shells there, along the lip of the road...."
...,...,...
145,Environment,"To pick a tulip from the garden, the red one. ..."
146,Environment,We are as clouds that veil the midnight moon; ...
147,Environment,"When pulled, the spider web took another form...."
148,Environment,Whose woods these are I think I know. His hous...


<class 'pandas.core.frame.DataFrame'>
Index: 991 entries, 0 to 149
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Genre   991 non-null    object
 1   Poem    987 non-null    object
dtypes: object(2)
memory usage: 23.2+ KB


None

In [4]:
print(data['Genre'].value_counts())
# Create a list of unique genres in the dataset
Name0=data['Genre'].unique().tolist()
# Sort the list of genre names
Name=sorted(Name0)

# Create a list of integers representing the genres
N=list(range(len(Name)))
# Create a mapping from genre names to integers
normal_mapping=dict(zip(Name,N)) 
# Create a reverse mapping from integers to genre names
reverse_mapping=dict(zip(N,Name)) 
# Map the 'Genre' column in the dataset to its corresponding integer representation
data['Genre']=data['Genre'].map(normal_mapping)

Genre
Environment    252
Music          250
Death          244
Affection      241
Name: count, dtype: int64


In [5]:
# Import necessary libraries
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import pandas as pd

# Define a class for vectorizing sequences using BERT
class BertSequenceVectorizer:
    def __init__(self):
        # Determine whether to use GPU or CPU
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        # Choose the BERT model name
        # Note: Two options are provided, choose based on internet connectivity
        #self.model_name = '../input/bert-base-uncased'  # Inet-not-connect
        self.model_name = 'bert-base-uncased'          # Inet-connect
        
        # Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        
        # Initialize the BERT model and move it to the selected device
        self.bert_model = BertModel.from_pretrained(self.model_name)
        self.bert_model = self.bert_model.to(self.device)
        
        # Set the maximum sequence length
        self.max_len = 128

    # Method to vectorize a given sentence using BERT
    def vectorize(self, sentence: str) -> np.array:
        # Tokenize the input sentence
        inp = self.tokenizer.encode(sentence)
        len_inp = len(inp)

        # Prepare inputs and masks for BERT
        if len_inp >= self.max_len:
            inputs = inp[:self.max_len]
            masks = [1] * self.max_len
        else:
            inputs = inp + [0] * (self.max_len - len_inp)
            masks = [1] * len_inp + [0] * (self.max_len - len_inp)

        # Convert inputs and masks to PyTorch tensors and move them to the selected device
        inputs_tensor = torch.tensor([inputs], dtype=torch.long).to(self.device)
        masks_tensor = torch.tensor([masks], dtype=torch.long).to(self.device)

        # Pass inputs and masks through the BERT model
        bert_out = self.bert_model(inputs_tensor, masks_tensor)
        seq_out, pooled_out = bert_out['last_hidden_state'], bert_out['pooler_output']

        # Convert the BERT output to a NumPy array and return
        if torch.cuda.is_available():    
            return seq_out[0][0].cpu().detach().numpy()
        else:
            return seq_out[0][0].detach().numpy()

# Create an instance of the BertSequenceVectorizer
BSV = BertSequenceVectorizer()

# Apply the vectorization function to each row in the 'Poem' column of the DataFrame
data['Poem_bert'] = data['Poem'].progress_apply(lambda x: BSV.vectorize(x))

# Display the first two rows of the DataFrame with the newly added 'Poem_bert' column
display(data[0:2])


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

100%|██████████| 987/987 [00:14<00:00, 66.21it/s]


,Genre,Poem,Poem_bert
1,3,In the thick brushthey spend the...,"[-0.1946835, 0.42541593, -0.26951808, -0.47039..."
2,3,Storms are generous. ...,"[-0.012234179, -0.33518994, -0.2274042, 0.0576..."


In [6]:
# Create an empty list to store BERT vectorized representations of poems
review = []

# Iterate over each item in the 'Poem_bert' column of the DataFrame
for item in data['Poem_bert']:
    # Append the BERT vectorized representation to the 'review' list
    review += [item]

# Create a DataFrame 'x' from the list of BERT vectorized representations
x = pd.DataFrame(review)

# Extract the 'Genre' column from the original data as the target variable 'y'
y = data['Genre']

# Print the shape of the feature matrix 'x' and the target variable 'y'
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

# Get the length of the original dataset 'data'
n = len(data)

# Create a list 'N' containing integers from 0 to (n-1)
N = list(range(n))

# Shuffle the list of integers to randomize the order
random.shuffle(N)

# The 'N' list now represents a shuffled order of indices for the original dataset
# You can use this shuffled order to create train/test splits or perform other operations


Shape of x: (987, 768)
Shape of y: (987,)


In [7]:
# Use train_test_split to split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Limit the number of features to the first 200 columns for both training and testing sets
x_train = x_train.iloc[:, 0:200]
x_test = x_test.iloc[:, 0:200]

# Convert the Pandas DataFrames to NumPy arrays for compatibility with machine learning libraries
x_train = np.array(x_train)
y_train = np.array(y_train)

x = np.array(x_train)
y = np.array(y_train)

# The resulting x_train, x_test, y_train, and y_test can now be used in machine learning models


In [8]:
# Create an instance of the LightGBM Regressor with specified hyperparameters
clf = lgb.LGBMRegressor(max_depth=6, n_estimators=1000, learning_rate=0.01)

# Create an instance of ShuffleSplit for cross-validation
# ShuffleSplit will create 5 splits, each with 80% training and 20% testing data
# random_state ensures reproducibility of the splits
ss = ShuffleSplit(n_splits=5, train_size=0.8, test_size=0.2, random_state=0)


In [9]:
# Reset the index of your DataFrame
data_reset = data.reset_index(drop=True)

# Iterate over the splits generated by ShuffleSplit
for train_index, test_index in ss.split(x):
    # Use the new DataFrame with the reset index
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Train the model
    clf.fit(x_train, y_train)
    
    # Print the model's score on the testing data
    print(clf.score(x_test, y_test))


0.13833853411978791
-0.022793300203905353
0.03731468009927519
-0.04670551165168191
-0.009088017872630916


In [10]:
# Use the trained LightGBM Regressor to predict the target variable on the testing data
y_pred = clf.predict(x_test)

In [11]:
# Initialize an empty list to store the rounded integer predictions
PRED = []

# Iterate over each prediction in y_pred
for pred in y_pred:
    # Round each prediction to the nearest integer and convert to int
    PRED += [int(round(pred, 0))]

# Convert the true labels from y_test to a list
ANS = list(y_test)

# Print the first 5 rounded predictions and true labels for comparison
print("Rounded Predictions:", PRED[0:5])
print("True Labels:", ANS[0:5])


Rounded Predictions: [1, 2, 2, 2, 1]
True Labels: [2, 2, 3, 1, 3]


In [12]:
# Convert the class names to strings if they are integers
target_names = [str(name) for name in Name]

# Use scikit-learn's classification_report to generate a report on the model's performance
# Print the classification report, including precision, recall, F1-score, and support for each class
# The 'target_names' parameter provides labels for each class in the report
# The 'digits' parameter specifies the number of digits to display
print(classification_report(ANS, PRED, target_names=target_names, digits=4))


              precision    recall  f1-score   support

   Affection     0.5000    0.0682    0.1200        44
       Death     0.2206    0.3947    0.2830        38
 Environment     0.2564    0.5263    0.3448        38
       Music     0.0000    0.0000    0.0000        38

    accuracy                         0.2405       158
   macro avg     0.2442    0.2473    0.1870       158
weighted avg     0.2540    0.2405    0.1844       158



In [13]:
# 1. Create a New Poem
new_poem = "Your Poem here"

# 2. Vectorize the New Poem
# Use the vectorize method of the BSV instance to get the BERT vector representation
new_poem_vector = BSV.vectorize(new_poem)

# Ensure that the vector has the same number of features as the model expects (e.g., first 200 columns)
new_poem_vector = new_poem_vector[:200]

# Convert the vector into a DataFrame for compatibility with the model
new_poem_df = pd.DataFrame([new_poem_vector])

# 3. Make Prediction
# Use the trained model to predict the genre for the new poem
predicted_genre = clf.predict(new_poem_df)

# Print the predicted genre
print("Predicted Genre:", predicted_genre)
# Round the predicted genre to the nearest integer
rounded_prediction = int(round(predicted_genre[0]))

# Print the rounded prediction
print("Rounded Predicted Genre:", rounded_prediction)



Predicted Genre: [1.35620857]
Rounded Predicted Genre: 1
